In [180]:
import geopandas as gpd
import pandas as pd


In [181]:
main_data_path = "/Users/test/Documents/code/iccs25/initial_data/Арктика/"


In [182]:
DEFAULT_CAPACITY_IF_NOT_FOUND = 600


In [183]:
SERVICE_AMENITIES_COUNT_COLUMN = "health"
SERVICE_NAME = "hospital"
SETTL_NAME = "nao"
TRANSPORT_COLS_WARM = [
    "car_warm",
    "plane",
    "water_ship",
    "water_boat",
]
TRANSPORT_COLS_COLD = [
    "car_cold",
    "plane",
    "winter_tr",
]  # later dropping plane. just because


Так короче тут делается следующим образом:
- населенные пункты по своему списку отличаются для зимы и лета
- поэтому тут с двух просто собираются все которые есть
- и там где нет данных доставляются с зимы кажется (или наоборот я не помню кого к кому присоединяют)

In [184]:
df_link = gpd.read_file(
    main_data_path + f"qgis/visualisation connections/connections_{SETTL_NAME}.gpkg"
)


In [185]:
df_link.head(2)


,settl1,settl2,car_warm,plane,water,car_cold,winter,geometry
0,Архангельск,Нарьян-Мар,True,True,False,True,False,"LINESTRING (5899871.342 10340576.088, 4511208...."
1,Архангельск,Несь,False,True,False,False,False,"LINESTRING (4973644.774 10043288.078, 4513811...."


In [186]:
df_link.explore()


In [187]:
df_time_warm = pd.read_csv(
    main_data_path + f"python/Расчёт времени/warm_time_{SETTL_NAME}.csv",
    sep=";",
    header=None,
)
df_time_warm.columns = [
    "edge1",
    "edge2",
] + TRANSPORT_COLS_WARM


In [188]:
msk = df_time_warm["edge1"] == "Нижняя Пеша"  # "Нижняя Пёша" - "Нижня Пёша"
df_time_warm[msk]


,edge1,edge2,car_warm,plane,water_ship,water_boat
92,Нижняя Пеша,Вижас,"0,00","0,42","0,00","0,00"
93,Нижняя Пеша,Индига,"0,00","0,58","0,00","0,00"
94,Нижняя Пеша,Чижа,"0,00","0,78","0,00","0,00"
95,Нижняя Пеша,Выучейский,"0,00","0,00","0,00","0,58"
96,Нижняя Пеша,Верхняя Пеша,"0,57","0,00","0,00","0,00"


In [189]:
msk = df_time_warm["edge2"] == "Нижняя Пеша"  # "Нижняя Пёша" - "Нижня Пёша"
df_time_warm[msk]


,edge1,edge2,car_warm,plane,water_ship,water_boat
6,Архангельск,Нижняя Пеша,"0,00","3,00","0,00","0,00"
53,Мезень,Нижняя Пеша,"0,00","0,00","0,00","0,00"
67,Нарьян-Мар,Нижняя Пеша,"0,00","1,27","0,00","0,00"
108,Ома,Нижняя Пеша,"0,00","0,27","0,00","0,00"


In [190]:
df_time_cold = pd.read_csv(
    main_data_path + f"python/Расчёт времени/cold_time_{SETTL_NAME}.csv",
    sep=";",
    header=None,
)
df_time_cold.columns = ["edge1", "edge2"] + TRANSPORT_COLS_COLD
# df_time_cold.drop(columns=["plane"], inplace=True)
TRANSPORT_COLS_COLD = ["car_cold", "winter_tr"]


In [191]:
df_time_warm.head(2)


,edge1,edge2,car_warm,plane,water_ship,water_boat
0,Андег,Нельмин-Нос,"0,00","0,98","0,83","0,83"
1,Андег,Куя,"0,00","0,00","0,75","0,75"


In [192]:
df_time_cold.head(2)


,edge1,edge2,car_cold,plane,winter_tr
0,Андег,Нельмин-Нос,"0,00","0,98","0,00"
1,Архангельск,Нарьян-Мар,"46,00","1,25","0,00"


In [193]:
df_all_data = gpd.read_file(main_data_path + f"{SETTL_NAME}_settl.geojson")
df_all_data = df_all_data[
    ["name", "population", SERVICE_AMENITIES_COUNT_COLUMN, "geometry"]
]
df_all_data = df_all_data.sort_values("name")


In [194]:
df_all_data["name"] = df_all_data["name"].str.replace("ё", "е")


In [195]:
df_all_data["population"] = (df_all_data["population"]).astype("Int64")


In [196]:
if SETTL_NAME == "nao":
    """
    Валерия сама руками добавила население тут
    """
    df_all_data.loc[df_all_data["name"] == "Архангельск", "population"] = 296665
    df_all_data.loc[df_all_data["name"] == "Воркута", "population"] = 54223
    df_all_data.loc[df_all_data["name"] == "Искателей", "population"] = 7253
    df_all_data.loc[df_all_data["name"] == "Красное", "population"] = 1625
    df_all_data.loc[df_all_data["name"] == "Куя", "population"] = 152
    df_all_data.loc[df_all_data["name"] == "Мезень", "population"] = 2832
    df_all_data.loc[df_all_data["name"] == "Нарьян-Мар", "population"] = 24266
    df_all_data.loc[df_all_data["name"] == "Усинск", "population"] = 31200
    df_all_data.loc[df_all_data["name"] == "Харьягинский", "population"] = 544

df_all_data.isnull().sum()


name          0
population    0
health        0
geometry      0
dtype: int64

In [197]:
# First, do the merge with suffixes
df_time = pd.merge(
    df_time_warm[["edge1", "edge2"] + TRANSPORT_COLS_WARM],
    df_time_cold[["edge1", "edge2"] + TRANSPORT_COLS_COLD],
    on=["edge1", "edge2"],
    how="outer",
    suffixes=("_warm", "_cold"),
).fillna(0)


In [198]:
df_time


,edge1,edge2,car_warm,plane,water_ship,water_boat,car_cold,winter_tr
0,Андег,Куя,"0,00","0,00","0,75","0,75",0,0
1,Андег,Нельмин-Нос,"0,00","0,98","0,83","0,83","0,00","0,00"
2,Архангельск,Индига,"0,00","4,00","0,00","0,00","0,00","0,00"
3,Архангельск,Мезень,"7,00","0,92","0,00","0,00","7,00","0,00"
4,Архангельск,Нарьян-Мар,"46,00","1,25","0,00","0,00","46,00","0,00"
...,...,...,...,...,...,...,...,...
122,Харута,Хорей-Вер,"0,00","0,63","0,00","0,00","0,00","0,00"
123,Хонгурей,Каменка,"0,50","0,00","0,00","0,00","0,00","1,00"
124,Чижа,Индига,"0,00","1,29","0,00","0,00","0,00","0,00"
125,Шойна,Чижа,"0,00","0,44","0,00","0,00","0,00","0,00"


In [199]:
# # Now clean up: for overlapping transport columns, keep only "_warm" values
# for col in TRANSPORT_COLS_WARM:
#     if col in TRANSPORT_COLS_COLD:
#         warm_col = f"{col}_warm"
#         cold_col = f"{col}_cold"
#         df_time[col] = df_time[warm_col].combine_first(df_time[cold_col])
#         df_time.drop([warm_col, cold_col], axis=1, inplace=True)
#     else:
#         df_time.rename(columns={f"{col}_warm": col}, inplace=True)

# for col in TRANSPORT_COLS_COLD:
#     if col not in TRANSPORT_COLS_WARM:
#         df_time.rename(columns={f"{col}_cold": col}, inplace=True)


# # settlements_df = settlements_df.drop('name', axis=1)


In [200]:
TRANSPORT_COLS_WARM + TRANSPORT_COLS_COLD


['car_warm', 'plane', 'water_ship', 'water_boat', 'car_cold', 'winter_tr']

In [201]:
df_time[TRANSPORT_COLS_WARM + TRANSPORT_COLS_COLD]


,car_warm,plane,water_ship,water_boat,car_cold,winter_tr
0,"0,00","0,00","0,75","0,75",0,0
1,"0,00","0,98","0,83","0,83","0,00","0,00"
2,"0,00","4,00","0,00","0,00","0,00","0,00"
3,"7,00","0,92","0,00","0,00","7,00","0,00"
4,"46,00","1,25","0,00","0,00","46,00","0,00"
...,...,...,...,...,...,...
122,"0,00","0,63","0,00","0,00","0,00","0,00"
123,"0,50","0,00","0,00","0,00","0,00","1,00"
124,"0,00","1,29","0,00","0,00","0,00","0,00"
125,"0,00","0,44","0,00","0,00","0,00","0,00"


In [202]:
TRANSPORT_COLS_WARM + TRANSPORT_COLS_COLD


['car_warm', 'plane', 'water_ship', 'water_boat', 'car_cold', 'winter_tr']

In [203]:
df_time.loc[:, TRANSPORT_COLS_WARM + TRANSPORT_COLS_COLD] = df_time[
    TRANSPORT_COLS_WARM + TRANSPORT_COLS_COLD
].apply(lambda x: x.str.replace(",", ".").astype("float32"), axis=1)
df_time.fillna(0, inplace=True)


/var/folders/jn/d6x3c7jd7594dtldxq22ysn80000gn/T/ipykernel_67493/3801848384.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_time.fillna(0, inplace=True)


In [204]:
df_time


,edge1,edge2,car_warm,plane,water_ship,water_boat,car_cold,winter_tr
0,Андег,Куя,0.0,0.0,0.75,0.75,0.0,0.0
1,Андег,Нельмин-Нос,0.0,0.98,0.83,0.83,0.0,0.0
2,Архангельск,Индига,0.0,4.0,0.0,0.0,0.0,0.0
3,Архангельск,Мезень,7.0,0.92,0.0,0.0,7.0,0.0
4,Архангельск,Нарьян-Мар,46.0,1.25,0.0,0.0,46.0,0.0
...,...,...,...,...,...,...,...,...
122,Харута,Хорей-Вер,0.0,0.63,0.0,0.0,0.0,0.0
123,Хонгурей,Каменка,0.5,0.0,0.0,0.0,0.0,1.0
124,Чижа,Индига,0.0,1.29,0.0,0.0,0.0,0.0
125,Шойна,Чижа,0.0,0.44,0.0,0.0,0.0,0.0


In [205]:
# Расчет среднего для каждой строки
df_time["mean_time"] = df_time[TRANSPORT_COLS_WARM + TRANSPORT_COLS_COLD].apply(
    lambda row: row[(row > 0)].mean(), axis=1
)


In [206]:
# df_time = df_time.dropna()
# df_time = df_time[["edge1", "edge2", "mean_time"]]


In [207]:
# Создаем копию датафрейма с обратным порядком edge1 и edge2
df_reversed = df_time.copy()
df_reversed[["edge1", "edge2"]] = df_reversed[["edge2", "edge1"]]

# Проверяем совпадения
matches = pd.merge(df_time, df_reversed, on=["edge1", "edge2"], how="inner")

if not matches.empty:
    print("Строки с обратным порядком найдены:")
else:
    print("Строк с обратным порядком не найдено.")

matches


Строки с обратным порядком найдены:


,edge1,edge2,car_warm_x,plane_x,water_ship_x,water_boat_x,car_cold_x,winter_tr_x,mean_time_x,car_warm_y,plane_y,water_ship_y,water_boat_y,car_cold_y,winter_tr_y,mean_time_y
0,Коткино,Лабожское,0.0,0.27,5.5,3.2,0.0,0.0,2.99,0.0,0.0,5.5,2.7,0.0,0.0,4.10
1,Лабожское,Коткино,0.0,0.0,5.5,2.7,0.0,0.0,4.10,0.0,0.27,5.5,3.2,0.0,0.0,2.99


In [208]:
if SETTL_NAME == "nao" and SERVICE_NAME == "hospital":
    df_time = df_time[
        ~((df_time["edge1"] == "Коткино") & (df_time["edge2"] == "Лабожское"))
    ]
    # raise Va


In [209]:
df_time = df_time[
    (df_time["edge1"].isin(df_all_data["name"].unique()))
    & (df_time["edge2"].isin(df_all_data["name"].unique()))
]


In [210]:
df_all_data["name"].nunique()


44

In [211]:
df_time


,edge1,edge2,car_warm,plane,water_ship,water_boat,car_cold,winter_tr,mean_time
0,Андег,Куя,0.0,0.0,0.75,0.75,0.0,0.0,0.750000
1,Андег,Нельмин-Нос,0.0,0.98,0.83,0.83,0.0,0.0,0.880000
2,Архангельск,Индига,0.0,4.0,0.0,0.0,0.0,0.0,4.000000
3,Архангельск,Мезень,7.0,0.92,0.0,0.0,7.0,0.0,4.973333
4,Архангельск,Нарьян-Мар,46.0,1.25,0.0,0.0,46.0,0.0,31.083333
...,...,...,...,...,...,...,...,...,...
122,Харута,Хорей-Вер,0.0,0.63,0.0,0.0,0.0,0.0,0.630000
123,Хонгурей,Каменка,0.5,0.0,0.0,0.0,0.0,1.0,0.750000
124,Чижа,Индига,0.0,1.29,0.0,0.0,0.0,0.0,1.290000
125,Шойна,Чижа,0.0,0.44,0.0,0.0,0.0,0.0,0.440000


In [212]:
list(df_link.columns)


['settl1',
 'settl2',
 'car_warm',
 'plane',
 'water',
 'car_cold',
 'winter',
 'geometry']

In [213]:
# 1. Соотнесение edge1 == settl1 и edge2 == settl2
df_time = pd.merge(
    df_time,
    df_link.loc[:, [col for col in df_link.columns if col not in df_time.columns]],
    left_on=["edge1", "edge2"],
    right_on=["settl1", "settl2"],
    how="left",
).drop(columns=["settl1", "settl2"])


# Удаляем лишние столбцы, если они не нужны
# df_time = df_time[["edge1", "edge2", "mean_time", "geometry"]]
df_time.isnull().sum()


edge1         0
edge2         0
car_warm      0
plane         0
water_ship    0
water_boat    0
car_cold      0
winter_tr     0
mean_time     0
water         2
winter        2
geometry      2
dtype: int64

In [214]:
df_time["mean_time"] = (df_time["mean_time"] * 60).round().astype("Int64")


In [215]:
# df_time = df_time[df_time["mean_time"] <= 420]


In [216]:
df_time = gpd.GeoDataFrame(df_time, geometry="geometry")


## Ну по mean_time конечно не совсем правильно смотреть наверное
### хотя если дропаем самолеты то как будто да тогда и оно меняется..

In [217]:
df_time[df_time["geometry"].notna()].explore(column="mean_time")


In [218]:
df_all_data.reset_index(inplace=True, drop=True)


In [219]:
if SERVICE_NAME == "hospital" and SETTL_NAME == "nao":
    """
    Валерия дособрала данные по больницам руками
    """
    df_all_data[SERVICE_NAME] = [
        0,
        0,
        12,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        4,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        3,
        0,
        1,
        1,
        1,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        1,
        0,
        0,
        1,
        0,
        0,
        0,
    ]
    df_all_data["capacity"] = df_all_data[SERVICE_NAME] * DEFAULT_CAPACITY_IF_NOT_FOUND

if "Архангельск" in df_all_data["name"].values:
    df_all_data.loc[df_all_data["name"] == "Архангельск", "capacity"] = 77021
    df_all_data.loc[df_all_data["name"] == "Воркута", "capacity"] = 20034
    df_all_data.loc[df_all_data["name"] == "Нарьян-Мар", "capacity"] = 14002
    df_all_data.loc[df_all_data["name"] == "Усинск", "capacity"] = 18057


In [220]:
# 1. Создание списка уникальных населенных пунктов
unique_settlements = sorted(set(df_time["edge1"]).union(set(df_time["edge2"])))

# 2. Создание словаря для отображения названий населенных пунктов в id
settlement_id_map = {name: idx for idx, name in enumerate(unique_settlements)}

# 3. Создание матрицы смежности с использованием id
adjacency_matrix = pd.DataFrame(
    0, index=range(len(unique_settlements)), columns=range(len(unique_settlements))
)

for _, row in df_time.iterrows():
    # for _, row in final_df[final_df['geometry'].notna()].iterrows():
    edge1, edge2 = row["edge1"], row["edge2"]
    id1, id2 = settlement_id_map[edge1], settlement_id_map[edge2]
    adjacency_matrix.at[id1, id2] = row["mean_time"]
    adjacency_matrix.at[id2, id1] = row["mean_time"]  # Граф ненаправленный

# 4. Создание датафрейма с уникальными населенными пунктами и их id
settlements_df = pd.DataFrame(
    {"id": range(len(unique_settlements)), "settlement": unique_settlements}
)


In [221]:
settlements_df


,id,settlement
0,0,Амдерма
1,1,Андег
2,2,Архангельск
3,3,Белушье
4,4,Бугрино
5,5,Великовисочное
6,6,Верхняя Пеша
7,7,Вижас
8,8,Волоковая
9,9,Волонга


In [222]:
df_all_data


,name,population,health,geometry,hospital,capacity
0,Амдерма,444,1,POINT (1070458.851 766094.225),0,0
1,Андег,175,1,POINT (834175.227 445156.698),0,0
2,Архангельск,296665,3,POINT (360029.734 -40535.595),12,77021
3,Белушье,50,1,POINT (633349.599 272766.98),0,0
4,Бугрино,416,1,POINT (651378.74 494278.85),0,0
5,Великовисочное,544,2,POINT (807485.193 362288.65),1,600
6,Верхняя Пеша,134,1,POINT (655584.56 245846.492),0,0
7,Вижас,75,1,POINT (565422.767 229163.324),0,0
8,Волоковая,105,1,POINT (670964.301 235090.958),0,0
9,Волонга,35,1,POINT (634026.056 300431.644),0,0


In [223]:
if "capacity" not in df_all_data.columns:
    df_all_data["capacity"] = DEFAULT_CAPACITY_IF_NOT_FOUND


In [224]:
service_df = pd.merge(
    settlements_df,
    df_all_data[["name", "population", "geometry", "capacity"]],
    left_on=["settlement"],
    right_on=["name"],
    how="left",
)
service_df = service_df[["id", "name", "population", "geometry", "capacity"]]


In [225]:
settlements_df = pd.merge(
    settlements_df,
    df_all_data[["name", "population", "geometry"]],
    left_on=["settlement"],
    right_on=["name"],
    how="left",
)
settlements_df = settlements_df[["id", "name", "population", "geometry"]]


In [226]:
df_time.to_file(f"df_time_{SETTL_NAME}.geojson", driver="GeoJSON")

settlements_df = gpd.GeoDataFrame(settlements_df, geometry="geometry")
settlements_df.to_file(f"df_settlements_{SETTL_NAME}.geojson", driver="GeoJSON")

service_df = gpd.GeoDataFrame(service_df, geometry="geometry")
service_df.to_file(f"df_{SERVICE_NAME}_{SETTL_NAME}.geojson", driver="GeoJSON")


In [227]:
service_df


,id,name,population,geometry,capacity
0,0,Амдерма,444,POINT (1070458.851 766094.225),0
1,1,Андег,175,POINT (834175.227 445156.698),0
2,2,Архангельск,296665,POINT (360029.734 -40535.595),77021
3,3,Белушье,50,POINT (633349.599 272766.98),0
4,4,Бугрино,416,POINT (651378.74 494278.85),0
5,5,Великовисочное,544,POINT (807485.193 362288.65),600
6,6,Верхняя Пеша,134,POINT (655584.56 245846.492),0
7,7,Вижас,75,POINT (565422.767 229163.324),0
8,8,Волоковая,105,POINT (670964.301 235090.958),0
9,9,Волонга,35,POINT (634026.056 300431.644),0


In [228]:
settlements_df


,id,name,population,geometry
0,0,Амдерма,444,POINT (1070458.851 766094.225)
1,1,Андег,175,POINT (834175.227 445156.698)
2,2,Архангельск,296665,POINT (360029.734 -40535.595)
3,3,Белушье,50,POINT (633349.599 272766.98)
4,4,Бугрино,416,POINT (651378.74 494278.85)
5,5,Великовисочное,544,POINT (807485.193 362288.65)
6,6,Верхняя Пеша,134,POINT (655584.56 245846.492)
7,7,Вижас,75,POINT (565422.767 229163.324)
8,8,Волоковая,105,POINT (670964.301 235090.958)
9,9,Волонга,35,POINT (634026.056 300431.644)


In [229]:
df_time


,edge1,edge2,car_warm,plane,water_ship,water_boat,car_cold,winter_tr,mean_time,water,winter,geometry
0,Андег,Куя,0.0,0.0,0.75,0.75,0.0,0.0,45,True,False,"LINESTRING (5924527.576 10414768.401, 5928876...."
1,Андег,Нельмин-Нос,0.0,0.98,0.83,0.83,0.0,0.0,53,True,False,"LINESTRING (5895204.715 10440994.24, 5924373.8..."
2,Архангельск,Индига,0.0,4.0,0.0,0.0,0.0,0.0,240,False,False,"LINESTRING (5458260.676 10345470.146, 4508605...."
3,Архангельск,Мезень,7.0,0.92,0.0,0.0,7.0,0.0,298,False,False,"LINESTRING (4925328.964 9836071.566, 4515477.6..."
4,Архангельск,Нарьян-Мар,46.0,1.25,0.0,0.0,46.0,0.0,1865,False,False,"LINESTRING (5899871.342 10340576.088, 4511208...."
...,...,...,...,...,...,...,...,...,...,...,...,...
121,Харута,Хорей-Вер,0.0,0.63,0.0,0.0,0.0,0.0,38,False,False,"LINESTRING (6627551.406 10110271.905, 6461849...."
122,Хонгурей,Каменка,0.5,0.0,0.0,0.0,0.0,1.0,45,False,True,"LINESTRING (5783503.285 10316904.725, 5775595...."
123,Чижа,Индига,0.0,1.29,0.0,0.0,0.0,0.0,77,False,False,"LINESTRING (4940696.169 10181108.03, 5457887.7..."
124,Шойна,Чижа,0.0,0.44,0.0,0.0,0.0,0.0,26,False,False,"LINESTRING (4915096.322 10411056.368, 4940751...."
